In [2]:
from datetime import datetime, time
from glob import glob
import re
from astropy.coordinates import SkyCoord
from astropy.table import Table
from astropy.time import Time
import astropy.units as u
import numpy as np

In [3]:
logs_dir = r"..\..\Files\DSSI Logs"
logs_to_process = glob(f"{logs_dir}/**/*.olist", recursive=True)

with open("DSSI observations.txt", mode="w") as obs:
    for file in logs_to_process:
        datestr = ''.join(re.findall("([0-9]{4})....(...[0-9]{2})", file)[0])
        utc_date = datetime.strptime(datestr, "%Y%b%d")
        print("========================================")
        print(file, utc_date)
        with open(file) as f:
            for line in f.readlines():
                if line.count(":") == 5: # and line.count("TIC") > 0:
                    obs.write(f"{datestr} {line}")
                    # print(line, end='')
        obs.write("-------\n")



..\..\Files\DSSI Logs\2022-Q3\sep27raw.olist 2022-09-27 00:00:00
..\..\Files\DSSI Logs\2022-Q3\sep28raw.olist 2022-09-28 00:00:00
..\..\Files\DSSI Logs\2022-Q3\sep29raw.olist 2022-09-29 00:00:00
..\..\Files\DSSI Logs\2022-Q3\sep30raw.olist 2022-09-30 00:00:00
..\..\Files\DSSI Logs\2022-Q4\nov12raw.olist 2022-11-12 00:00:00
..\..\Files\DSSI Logs\2022-Q4\nov13raw.olist 2022-11-13 00:00:00
..\..\Files\DSSI Logs\2022-Q4\nov14raw.olist 2022-11-14 00:00:00
..\..\Files\DSSI Logs\2022-Q4\nov15raw.olist 2022-11-15 00:00:00
..\..\Files\DSSI Logs\2022-Q4\nov16raw.olist 2022-11-16 00:00:00
..\..\Files\DSSI Logs\2023-Q1\mar07raw.olist 2023-03-07 00:00:00
..\..\Files\DSSI Logs\2023-Q1\mar08raw.olist 2023-03-08 00:00:00
..\..\Files\DSSI Logs\2023-Q1\mar09raw.olist 2023-03-09 00:00:00
..\..\Files\DSSI Logs\2023-Q1\mar11raw.olist 2023-03-11 00:00:00
..\..\Files\DSSI Logs\2023-Q1\mar12raw.olist 2023-03-12 00:00:00
..\..\Files\DSSI Logs\2023-Q2\may09raw.olist 2023-05-09 00:00:00
..\..\Files\DSSI Logs\202

Next steps:
  * Edit the produced `DSSI observations.txt` file in Notepad++ to align the columns properly.
  * Remove ~230 lines that are in a different format.
    * Some are different only because the target name is longer.
    * Some represent optical + infrared measurements and therefore have a range of image numbers for optical and a single image number for infrared.
  * Rename resulting file to `DSSI observations.csv` so manual work doesn't get clobbered if code is rerun.

In [32]:
observations = Table.read("DSSI observations.csv")
tic_observations = observations[np.char.find(observations["Notes"], "TIC") >= 0]

tic_observations = tic_observations[~(tic_observations["Name"] == "H300111")] #bad time format

tic_observations["DateTimeUtc"] = None
tic_observations["TIC ID"] = None
tic_observations["BYear"] = None

for row in tic_observations:
    datetimeutc = datetime.strptime(row["Date"]+row["Time"], "%Y%b%d%H:%M")
    row["DateTimeUtc"] = datetimeutc
    row["BYear"] = Time(datetimeutc, format="datetime").byear
    row["TIC ID"] = "TIC " + re.findall("[#]*TIC[ ]*[ID ]*[=]*[ ]*([0-9]+)", row["Notes"])[0]
    coord = SkyCoord(ra=row["RA"], dec=row["Dec"], unit=[u.hourangle, u.deg])
    row["RA"] = coord.ra
    row["Dec"] = coord.dec

tic_observations.remove_columns(["Date", "Time", "Notes"])
tic_observations.write("TIC observations.csv", overwrite=True)
tic_observations

c:\Users\User\anaconda3\envs\Astro\Lib\site-packages\astropy\table\row.py:71: StringTruncateWarning: truncated right side string(s) longer than 13 character(s) during assignment
  self._table.columns[item][self._index] = val
c:\Users\User\anaconda3\envs\Astro\Lib\site-packages\astropy\table\row.py:71: StringTruncateWarning: truncated right side string(s) longer than 13 character(s) during assignment
  self._table.columns[item][self._index] = val
c:\Users\User\anaconda3\envs\Astro\Lib\site-packages\astropy\table\row.py:71: StringTruncateWarning: truncated right side string(s) longer than 13 character(s) during assignment
  self._table.columns[item][self._index] = val
c:\Users\User\anaconda3\envs\Astro\Lib\site-packages\astropy\table\row.py:71: StringTruncateWarning: truncated right side string(s) longer than 13 character(s) during assignment
  self._table.columns[item][self._index] = val
c:\Users\User\anaconda3\envs\Astro\Lib\site-packages\astropy\table\row.py:71: StringTruncateWarning:

Name,ImageNum,Gain1,Gain2,RA,Dec,PMRA,PMDec,Mag,DateTimeUtc,TIC ID,BYear
str7,int32,int32,int32,str10,str13,float64,float64,float64,object,object,object
H300032,91,300,300,8.85125,62.9015833333,-2.727,-0.154,11.73,2022-09-28 08:22:00,TIC 283940788,2022.7419373794528
H300032,92,300,300,8.85125,62.9015833333,-2.727,-0.154,11.73,2022-09-28 08:23:00,TIC 283940788,2022.7419392807785
H300032,93,300,300,8.85125,62.9015833333,-2.727,-0.154,11.73,2022-09-28 08:25:00,TIC 283940788,2022.7419430834302
H300032,94,300,300,8.85125,62.9015833333,-2.727,-0.154,11.73,2022-09-28 08:26:00,TIC 283940788,2022.7419449847562
H300032,95,300,300,8.85125,62.9015833333,-2.727,-0.154,11.73,2022-09-28 08:28:00,TIC 283940788,2022.7419487874079
H300033,97,300,300,11.86375,64.8180277777,-2.559,-0.239,12.02,2022-09-28 08:36:00,TIC 284814380,2022.7419639980149
H300033,98,300,300,11.86375,64.8180277777,-2.559,-0.239,12.02,2022-09-28 08:37:00,TIC 284814380,2022.7419658993408
H300033,99,300,300,11.86375,64.8180277777,-2.559,-0.239,12.02,2022-09-28 08:39:00,TIC 284814380,2022.7419697019927
H300033,100,300,300,11.86375,64.8180277777,-2.559,-0.239,12.02,2022-09-28 08:41:00,TIC 284814380,2022.7419735046444


In [11]:
asd = datetime.strptime("2022-09-28 08:22:00", "%Y-%m-%d %H:%M:%S")
print(asd, type(asd))
Time(asd, format="datetime").jd

2022-09-28 08:22:00 <class 'datetime.datetime'>


2459850.8486111113